## Manually label and upload photos to the database

* **Input:** folder with classifc image classification format:

```
images/
    class_1/
        image_1.jpg
        image_2.jpg
        ...
    class_2/
        image_3.jpeg
        image_4.jpeg
        ...
    ...
```

* **Output:** Labelled images stored in GCP and tracked with Weights & Biases Artifacts. 


In [1]:
# Append the upper level directory to sys
import sys
sys.path.append("..")

import pandas as pd
import numpy as np

from pathlib import Path

In [2]:
# !rm -rf food_photos/*
# !rm -rf _MACOSX
# !rm -rf _MACOSX/*

In [3]:
# !unzip -q 2023-02-15-food_photos.zip

In [4]:
# Get list of all paths in food_photos directory
path = Path('food_photos')
all_paths = sorted(list(path.glob('*/*.jp*g')))
all_paths[:10]

[PosixPath('food_photos/banana/1c91d84f-3161-4c20-bd89-1af5199099af.jpeg'),
 PosixPath('food_photos/banana/624c1c45-a265-4046-905a-fec5f4f51869.jpeg'),
 PosixPath('food_photos/banana/88618b3a-8a19-4fd2-b4a6-54f8c0b8c29f.jpeg'),
 PosixPath('food_photos/banana/b18ddefa-8d7e-4a4d-af06-d3575179e9d9.jpeg'),
 PosixPath('food_photos/banana/dab2fdf4-7719-4809-9132-4253e84476f8.jpeg'),
 PosixPath('food_photos/banana/e302ade1-f8d1-4751-bc99-ab3e998204f3.jpeg'),
 PosixPath('food_photos/banana/f777e9f8-c07b-407a-9fcd-d093f08dd738.jpeg'),
 PosixPath('food_photos/biltong/58534371-a235-4608-a255-4a3468d6e78b.jpeg'),
 PosixPath('food_photos/blueberries/7985df7f-ce68-4390-acb1-8a18e71e8816.jpeg'),
 PosixPath('food_photos/broccolini/00b1a7d6-1dc8-4f8c-a7c4-537eea2f0936.jpeg')]

In [5]:
len(all_paths)

1381

In [6]:
import uuid

def is_valid_uuid(string):
    try:
        uuid.UUID(string)
        return True
    except ValueError:
        return False

filename = "some-file-name"
if is_valid_uuid(filename):
    print(f"{filename} is a valid UUID")
else:
    print(f"{filename} is not a valid UUID")

some-file-name is not a valid UUID


In [7]:
# Loop through all_paths, if the path is a UUID, pass, if not, rename the file to a UUID
for path in all_paths:
    if not is_valid_uuid(path.stem):
        print(f"[INFO] {path} isn't valud UUID, renaming to UUID...")
        new_name = f"{uuid.uuid4()}.jpeg"
        new_path_name = path.parent / new_name
        path.rename(new_path_name)
        print(f"[INFO] Renamed {path} to {new_path_name}")

# Get list of all paths in food_photos directory
path = Path('food_photos')
all_paths = sorted(list(path.glob('*/*.jpeg')))
all_paths[:10]

[PosixPath('food_photos/banana/1c91d84f-3161-4c20-bd89-1af5199099af.jpeg'),
 PosixPath('food_photos/banana/624c1c45-a265-4046-905a-fec5f4f51869.jpeg'),
 PosixPath('food_photos/banana/88618b3a-8a19-4fd2-b4a6-54f8c0b8c29f.jpeg'),
 PosixPath('food_photos/banana/b18ddefa-8d7e-4a4d-af06-d3575179e9d9.jpeg'),
 PosixPath('food_photos/banana/dab2fdf4-7719-4809-9132-4253e84476f8.jpeg'),
 PosixPath('food_photos/banana/e302ade1-f8d1-4751-bc99-ab3e998204f3.jpeg'),
 PosixPath('food_photos/banana/f777e9f8-c07b-407a-9fcd-d093f08dd738.jpeg'),
 PosixPath('food_photos/biltong/58534371-a235-4608-a255-4a3468d6e78b.jpeg'),
 PosixPath('food_photos/blueberries/7985df7f-ce68-4390-acb1-8a18e71e8816.jpeg'),
 PosixPath('food_photos/broccolini/00b1a7d6-1dc8-4f8c-a7c4-537eea2f0936.jpeg')]

In [8]:
# Find the number of .jpeg images in each subfolder of food_photos and create a dictionary mapping the subfolder name to the number of images
subfolder_to_num_images = {}
for p in all_paths:
    subfolder = p.parent.stem
    if subfolder in subfolder_to_num_images:
        subfolder_to_num_images[subfolder] += 1
    else:
        subfolder_to_num_images[subfolder] = 1
subfolder_to_num_images

{'banana': 7,
 'biltong': 1,
 'blueberries': 1,
 'broccolini': 145,
 'brussel_sprouts': 4,
 'cashews': 5,
 'cauliflower': 1,
 'celery': 15,
 'cherries': 4,
 'chilli': 112,
 'coriander': 2,
 'corn': 133,
 'cucumber': 1,
 'fennel': 137,
 'fruit_smoothie': 2,
 'garlic': 2,
 'grapes_black': 4,
 'grapes_red': 165,
 'grapes_white': 4,
 'green_beans': 1,
 'ice_coffee': 4,
 'lemon': 1,
 'lychee': 165,
 'milk': 2,
 'omelette': 1,
 'onion_green': 128,
 'orange_juice': 2,
 'parsley': 136,
 'pear': 147,
 'prosciutto': 1,
 'radish': 4,
 'red_wine': 2,
 'salami': 1,
 'steak': 5,
 'sushi': 1,
 'tea': 5,
 'tomato': 29,
 'yoghurt': 1}

In [9]:
len(subfolder_to_num_images)

38

In [10]:
# Get all food_types from all_paths
food_types = sorted(list(set([p.parent.name for p in all_paths])))
food_types

['banana',
 'biltong',
 'blueberries',
 'broccolini',
 'brussel_sprouts',
 'cashews',
 'cauliflower',
 'celery',
 'cherries',
 'chilli',
 'coriander',
 'corn',
 'cucumber',
 'fennel',
 'fruit_smoothie',
 'garlic',
 'grapes_black',
 'grapes_red',
 'grapes_white',
 'green_beans',
 'ice_coffee',
 'lemon',
 'lychee',
 'milk',
 'omelette',
 'onion_green',
 'orange_juice',
 'parsley',
 'pear',
 'prosciutto',
 'radish',
 'red_wine',
 'salami',
 'steak',
 'sushi',
 'tea',
 'tomato',
 'yoghurt']

## Download original labels from Weights & Biases

In [11]:
# Get config
from configs.default_config import config

args = config
print(args)

namespace(annotations_columns_to_export=['filename', 'image_name', 'class_name', 'label', 'split', 'clear_or_confusing', 'whole_food_or_dish', 'one_food_or_multiple', 'label_last_updated_at', 'label_source', 'image_source'], auto_augment=True, batch_size=128, epochs=10, gs_bucket_name='food_vision_bucket_with_object_versioning', gs_image_storage_path='https://storage.cloud.google.com/food_vision_bucket_with_object_versioning/all_images/', input_size=224, label_smoothing=0.1, learning_rate=0.001, model='coatnext_nano_rw_224', num_to_try_and_autocorrect=1000, path_to_gcp_credentials='utils/google-storage-key.json', path_to_label_studio_api_key='utils/label_studio_api_key.json', pretrained=True, seed=42, use_mixed_precision=True, wandb_dataset_artifact='food_vision_199_classes_images:latest', wandb_job_type='', wandb_labels_artifact='food_vision_labels:latest', wandb_model_artifact='trained_model:latest', wandb_project='test_wandb_artifacts_by_reference', wandb_run_notes='', wandb_run_tag

In [12]:
# Connect to GCP
from utils.gcp_utils import set_gcp_credentials, test_gcp_connection
set_gcp_credentials(path_to_key="../utils/google-storage-key.json")
test_gcp_connection()

[INFO] GCP credentials set!
[INFO] GCP connection successful! Access to GCP for saving/loading data and models available.


In [13]:
import wandb

# Initialize a new run
from utils.wandb_utils import wandb_load_artifact, wandb_download_and_load_labels

notes = f"add {len(all_paths)} manually taken photos to the training dataset"

run = wandb.init(project=args.wandb_project, 
                 job_type=args.wandb_job_type,
                 tags=['manual_photo_upload'],
                 notes=notes)

annotations, class_names, class_dict, reverse_class_dict, labels_path = wandb_download_and_load_labels(wandb_run=run,
wandb_labels_artifact_name=args.wandb_labels_artifact)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mrdbourke. Use `wandb login --relogin` to force relogin


[INFO] Labels directory: ./artifacts/food_vision_labels:v26
[INFO] Labels path: artifacts/food_vision_labels:v26/annotations.csv
[INFO] Working with: 208 classes


In [18]:
len(annotations.label.unique())

208

In [110]:
# Make a copy of the annotations
original_annotations = annotations.copy()

In [111]:
# Get the columns of the annotations
columns_to_create = list(original_annotations.columns)
columns_to_create

['filename',
 'image_name',
 'class_name',
 'label',
 'split',
 'clear_or_confusing',
 'whole_food_or_dish',
 'one_food_or_multiple',
 'label_last_updated_at',
 'label_source',
 'image_source']

In [112]:
# Get all the image paths from food_photos
image_paths = list(Path('food_photos').glob('*/*.jpeg'))
len(image_paths)

1381

In [113]:
# Check to see what food_types are in class_names
food_types_in_class_names = [food_type for food_type in food_types if food_type in class_names]

# Check to see what food_types are not in class_names
food_types_not_in_class_names = [food_type for food_type in food_types if food_type not in class_names]
len(food_types_in_class_names)

33

In [114]:
food_types_in_class_names

['banana',
 'biltong',
 'blueberries',
 'broccolini',
 'brussel_sprouts',
 'cashews',
 'cauliflower',
 'celery',
 'cherries',
 'chilli',
 'coriander',
 'corn',
 'cucumber',
 'fennel',
 'garlic',
 'green_beans',
 'ice_coffee',
 'lemon',
 'lychee',
 'milk',
 'omelette',
 'onion_green',
 'orange_juice',
 'parsley',
 'pear',
 'radish',
 'red_wine',
 'salami',
 'steak',
 'sushi',
 'tea',
 'tomato',
 'yoghurt']

In [115]:
food_types_not_in_class_names

['fruit_smoothie', 'grapes_black', 'grapes_red', 'grapes_white', 'prosciutto']

In [116]:
# How to slot new classes into existing class_dict?
# Then update the existing annotations with the new class labels (e.g. "apple_green" -> 2, "apple_red" -> 3)

# Add new classes to class_dict
for food_type in food_types_not_in_class_names:
    class_dict[len(class_dict)] = food_type
    class_names.append(food_type)
    reverse_class_dict[food_type] = len(class_dict)-1

In [117]:
len(class_dict)

209

In [118]:
from utils.misc import get_now_time

# Create a list of dictionaries and fill out the columns of the annotations
columns = ['filename',
 'image_name',
 'class_name',
 'label',
 'split',
 'clear_or_confusing',
 'whole_food_or_dish',
 'one_food_or_multiple',
 'label_last_updated_at',
 'label_source',
 'image_source']

food_image_dict_list = []

for image_path in image_paths:
    food_image_dict = {}
    food_image_dict['filename'] = image_path
    food_image_dict['image_name'] = image_path.name
    food_image_dict['class_name'] = image_path.parent.name
    food_image_dict['label'] = reverse_class_dict[image_path.parent.name]
    # food_image_dict['split'] = 'train'
    # Label 20% of the images as test
    if np.random.random() < 0.2:
        food_image_dict['split'] = 'test'
    else:
        food_image_dict['split'] = 'train'
    food_image_dict['clear_or_confusing'] = 'clear'
    food_image_dict['whole_food_or_dish'] = 'whole_food'
    food_image_dict['one_food_or_multiple'] = 'one_food'
    food_image_dict['label_last_updated_at'] = get_now_time()
    food_image_dict['label_source'] = 'manual_upload'
    food_image_dict['image_source'] = 'manual_upload'

    food_image_dict_list.append(food_image_dict)

# Create a dataframe from the list of dictionaries
import pandas as pd
new_annotations = pd.DataFrame(food_image_dict_list)
new_annotations.head()

,filename,image_name,class_name,label,split,clear_or_confusing,whole_food_or_dish,one_food_or_multiple,label_last_updated_at,label_source,image_source
0,food_photos/cashews/96237783-2f60-4f2d-983a-f1...,96237783-2f60-4f2d-983a-f1b289067887.jpeg,cashews,36,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
1,food_photos/cashews/3ab541fe-7b51-4382-acaa-98...,3ab541fe-7b51-4382-acaa-98e895dd995f.jpeg,cashews,36,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
2,food_photos/cashews/4ae82b11-052f-4b2f-b2f8-da...,4ae82b11-052f-4b2f-b2f8-da4054f6d03f.jpeg,cashews,36,test,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
3,food_photos/cashews/c6d56f23-4aee-4e35-a1a7-6b...,c6d56f23-4aee-4e35-a1a7-6b7a9cf66600.jpeg,cashews,36,test,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
4,food_photos/cashews/80ca8d9c-bf05-49af-84ef-6f...,80ca8d9c-bf05-49af-84ef-6f25a15241af.jpeg,cashews,36,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload


In [119]:
# Print how many images are train/test in new_annotations
new_annotations['split'].value_counts()

train    1100
test      281
Name: split, dtype: int64

In [123]:
# Find a random sample of 10 images where the label value is 199 or higher
new_annotations[new_annotations['label'] >= 199].sample(10)

,filename,image_name,class_name,label,split,clear_or_confusing,whole_food_or_dish,one_food_or_multiple,label_last_updated_at,label_source,image_source
150,food_photos/grapes_red/9b7d41a7-efd3-4d46-9041...,9b7d41a7-efd3-4d46-9041-383778358bab.jpeg,grapes_red,206,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
302,food_photos/lychee/4b635799-ae6c-454c-821c-0a0...,4b635799-ae6c-454c-821c-0a0f7a6e7b77.jpeg,lychee,202,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
98,food_photos/grapes_red/f83830f0-dc48-4c62-bcc4...,f83830f0-dc48-4c62-bcc4-d5528dda20c2.jpeg,grapes_red,206,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
73,food_photos/grapes_red/867e4bb4-2bcd-495d-a22c...,867e4bb4-2bcd-495d-a22c-6789e0263adc.jpeg,grapes_red,206,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
131,food_photos/grapes_red/dd5d3a55-c645-4373-b657...,dd5d3a55-c645-4373-b657-4a953cbfdf94.jpeg,grapes_red,206,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
614,food_photos/fruit_smoothie/1f4d26a6-cb28-46ab-...,1f4d26a6-cb28-46ab-8ed6-0fa7fe31e7bf.jpeg,fruit_smoothie,204,test,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
155,food_photos/grapes_red/d6640599-e55d-4a33-bf3c...,d6640599-e55d-4a33-bf3c-19c7a830af97.jpeg,grapes_red,206,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
30,food_photos/grapes_red/613af66e-b1cf-46a0-8cd6...,613af66e-b1cf-46a0-8cd6-36e32d5b08a6.jpeg,grapes_red,206,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
126,food_photos/grapes_red/1b79b584-a31e-464f-807d...,1b79b584-a31e-464f-807d-7ffa70995d56.jpeg,grapes_red,206,test,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
296,food_photos/lychee/7454ab6f-fb2a-45d5-80de-94f...,7454ab6f-fb2a-45d5-80de-94fac5a192e5.jpeg,lychee,202,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload


In [124]:
len(original_annotations), len(new_annotations)

(38886, 1381)

In [125]:
# Append the new_annotations to the original_annotations
updated_annotations = pd.concat([original_annotations, new_annotations], ignore_index=True)
# updated_annotations = original_annotations.(new_annotations, ignore_index=True)
updated_annotations

,filename,image_name,class_name,label,split,clear_or_confusing,whole_food_or_dish,one_food_or_multiple,label_last_updated_at,label_source,image_source
0,test/pain_au_chocolat/4fd7cb42-bd7f-48f1-bfdc-...,4fd7cb42-bd7f-48f1-bfdc-607c2f54b788.jpg,pain_au_chocolat,121,test,NaN,NaN,NaN,NaN,NaN,internet_download
1,test/pain_au_chocolat/2062f52a-781c-4e4f-b8a7-...,2062f52a-781c-4e4f-b8a7-0a108934f453.jpg,pain_au_chocolat,121,test,NaN,NaN,NaN,NaN,NaN,internet_download
2,test/pain_au_chocolat/8003e0f6-37e8-460d-9c14-...,8003e0f6-37e8-460d-9c14-e7c6fe44a37f.jpg,pain_au_chocolat,121,test,NaN,NaN,NaN,NaN,NaN,internet_download
3,test/pain_au_chocolat/839437c8-c643-408f-9f04-...,839437c8-c643-408f-9f04-d0d3bec238c3.jpg,pain_au_chocolat,121,test,NaN,NaN,NaN,NaN,NaN,internet_download
4,test/pain_au_chocolat/ca5c13ff-a535-4b69-9144-...,ca5c13ff-a535-4b69-9144-e06275e01e35.jpg,pain_au_chocolat,121,test,NaN,NaN,NaN,NaN,NaN,internet_download
...,...,...,...,...,...,...,...,...,...,...,...
40262,food_photos/prosciutto/3adcda2f-6bc0-495a-ae37...,3adcda2f-6bc0-495a-ae37-398c5d72e646.jpeg,prosciutto,208,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
40263,food_photos/ice_coffee/c1ab14a3-9ebe-418e-9724...,c1ab14a3-9ebe-418e-9724-c1045e5db0fb.jpeg,ice_coffee,89,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
40264,food_photos/ice_coffee/f127961a-5444-4041-b0fd...,f127961a-5444-4041-b0fd-6e049f683853.jpeg,ice_coffee,89,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload
40265,food_photos/ice_coffee/1e3ead6d-71fe-4ebf-af09...,1e3ead6d-71fe-4ebf-af09-ffbc25cc5773.jpeg,ice_coffee,89,train,clear,whole_food,one_food,2023-02-15_13-51-57,manual_upload,manual_upload


In [126]:
# How many unique class_names are in original_annotations?
len(original_annotations['class_name'].unique())

204

In [127]:
# How many unique class_names are in updated_annotations?
len(updated_annotations['class_name'].unique())

209

In [128]:
# Check the difference in lengths between the original_annotations and updated_annotations
num_differences = len(updated_annotations) - len(original_annotations)
num_differences

1381

In [129]:
# Upload the updated annotations to Google Storage and track the changes
from utils.gcp_utils import upload_to_gs, rename_blob, delete_blob
from utils.wandb_utils import wandb_add_artifact_with_reference
from utils.misc import get_now_time
import os

GS_BUCKET_NAME = config.gs_bucket_name

UPDATED_ANNOTATIONS_TARGET_FILENAME = "updated_annotations.csv"
ORIGINAL_ANNOTATIONS_TARGET_FILENAME = "annotations.csv"

# Export the updated annotations to a CSV
columns_to_export = config.annotations_columns_to_export
print(f"[INFO] Exporting the following columns to {UPDATED_ANNOTATIONS_TARGET_FILENAME}: {columns_to_export}")

# TODO: Check if the updated_annotations_reset_index and the original_annotations actually differ, if so save them and upload them, else exit
if num_differences > 0:
    print(f"[INFO] {num_differences} changes to annotations.csv, updated label files and original annotations are different, saving the updated annotations.csv")

    # Export the updated_annotations_reset_index to a csv
    updated_annotations[columns_to_export].to_csv(UPDATED_ANNOTATIONS_TARGET_FILENAME, index=False)

    # Upload the updated CSV to Google Storage
    upload_to_gs(bucket_name=GS_BUCKET_NAME, 
                 source_file_name=UPDATED_ANNOTATIONS_TARGET_FILENAME, 
                 destination_blob_name=UPDATED_ANNOTATIONS_TARGET_FILENAME)

    # Rename the old CSV on Google Storage
    bucket_to_move_old_annotations_to = "old_annotations"
    name_to_rename_old_annotations = os.path.join(bucket_to_move_old_annotations_to, f"{get_now_time()}_old_annotations.csv")

    rename_blob(bucket_name=GS_BUCKET_NAME,
                blob_name=ORIGINAL_ANNOTATIONS_TARGET_FILENAME,
                new_name=name_to_rename_old_annotations)

    # Rename the "updated_annotations.csv" on Google Storage to "annotations.csv" 
    rename_blob(bucket_name=GS_BUCKET_NAME,
                blob_name=UPDATED_ANNOTATIONS_TARGET_FILENAME,
                new_name=ORIGINAL_ANNOTATIONS_TARGET_FILENAME)

    # Track the changes in the annotations with Weights & Biases
    annotations_path_on_gcs = f"gs://{GS_BUCKET_NAME}/{ORIGINAL_ANNOTATIONS_TARGET_FILENAME}"
    wandb_add_artifact_with_reference(wandb_run=run,
                                      artifact_name="food_vision_labels",
                                      artifact_type="labels",
                                      description="Labels for FoodVision project",
                                      reference_path=annotations_path_on_gcs)
else:
    print("[INFO] No changes to annotations.csv, updated label files and original annotations are the same, try fixing/updating the label files and try again")

[INFO] Exporting the following columns to updated_annotations.csv: ['filename', 'image_name', 'class_name', 'label', 'split', 'clear_or_confusing', 'whole_food_or_dish', 'one_food_or_multiple', 'label_last_updated_at', 'label_source', 'image_source']
[INFO] 1381 changes to annotations.csv, updated label files and original annotations are different, saving the updated annotations.csv
[INFO] Uploading updated_annotations.csv to updated_annotations.csv...
[INFO] Connected to Google Storage bucket: food_vision_bucket_with_object_versioning
[INFO] File updated_annotations.csv uploaded to food_vision_bucket_with_object_versioning/updated_annotations.csv.
[INFO] File size: 6699507 bytes
[INFO] Blob annotations.csv has been renamed to old_annotations/2023-02-15_13-54-10_old_annotations.csv
[INFO] Blob updated_annotations.csv has been renamed to annotations.csv
[INFO] Logging 'food_vision_labels' from 'gs://food_vision_bucket_with_object_versioning/annotations.csv' to Weights & Biases...


## Upload all photos in food_photos to Google Storage

In [130]:
from utils.gcp_utils import upload_to_gs, get_list_of_blobs

# Get list of images already in GCP bucket
gs_image_paths = get_list_of_blobs(bucket_name=GS_BUCKET_NAME, prefix="all_images")
print(f"[INFO] There are {len(gs_image_paths)} images in the GCP bucket")

[INFO] There are 38009 images in the GCP bucket


In [131]:
gs_image_paths_name = [str(blob.name).split("/")[-1] for blob in gs_image_paths]
gs_image_paths_name[:10]

['00045a69-b09f-4293-8c2e-a7ba27964fb6.jpg',
 '0009bea1-1577-4db5-bc60-b27eaed2d276.jpg',
 '000bd61c-f25c-415c-8cf1-3adc2d788320.jpg',
 '000cb663-06ac-4f96-8f9b-795cd05b5bcf.jpg',
 '000ec632-ee17-4988-9b32-8e4cb1421636.jpg',
 '0010ae4e-0879-4447-9610-e9d422caadd7.jpg',
 '0012b344-1bf9-421b-977e-d4ad8a6dcb7b.jpg',
 '00141b2a-0ed8-45b0-8823-8c0dbef3ac93.jpg',
 '0014f5f6-c37f-4922-8524-efc118f7d17b.jpg',
 '0018f7dd-217b-478b-9858-289922cf9715.jpg']

In [134]:
# Upload the images to Google Storage
!gsutil -m cp -r food_photos/*/*.jpeg gs://$GS_BUCKET_NAME/all_images/

Copying file://food_photos/banana/1c91d84f-3161-4c20-bd89-1af5199099af.jpeg [Content-Type=image/jpeg]...
Copying file://food_photos/banana/624c1c45-a265-4046-905a-fec5f4f51869.jpeg [Content-Type=image/jpeg]...
Copying file://food_photos/banana/88618b3a-8a19-4fd2-b4a6-54f8c0b8c29f.jpeg [Content-Type=image/jpeg]...
Copying file://food_photos/banana/b18ddefa-8d7e-4a4d-af06-d3575179e9d9.jpeg [Content-Type=image/jpeg]...
Copying file://food_photos/banana/dab2fdf4-7719-4809-9132-4253e84476f8.jpeg [Content-Type=image/jpeg]...
Copying file://food_photos/banana/e302ade1-f8d1-4751-bc99-ab3e998204f3.jpeg [Content-Type=image/jpeg]...
Copying file://food_photos/banana/f777e9f8-c07b-407a-9fcd-d093f08dd738.jpeg [Content-Type=image/jpeg]...
Copying file://food_photos/broccolini/0230a356-23fe-4ddd-9015-b396540608b3.jpeg [Content-Type=image/jpeg]...
Copying file://food_photos/broccolini/03db9365-7cc1-4703-b55a-48dc3fc50a7a.jpeg [Content-Type=image/jpeg]...
Copying file://food_photos/blueberries/7985df7f

In [135]:
# # Test uploading of 1 image
# one_image = image_paths[0]

# # Upload the image to Google Storage
# upload_to_gs(bucket_name=GS_BUCKET_NAME,
#              source_file_name=one_image,
#              destination_blob_name=f"all_images/{one_image.name}")

In [136]:
# Get list of images already in GCP bucket
gs_image_paths = get_list_of_blobs(bucket_name=GS_BUCKET_NAME, prefix="all_images")
print(f"[INFO] There are {len(gs_image_paths)} images in the GCP bucket")

[INFO] There are 39390 images in the GCP bucket


In [137]:
gs_image_paths_name = [str(blob.name).split("/")[-1] for blob in gs_image_paths]
gs_image_paths_name[:10]

['00045a69-b09f-4293-8c2e-a7ba27964fb6.jpg',
 '0009bea1-1577-4db5-bc60-b27eaed2d276.jpg',
 '000bd61c-f25c-415c-8cf1-3adc2d788320.jpg',
 '000cb663-06ac-4f96-8f9b-795cd05b5bcf.jpg',
 '000ec632-ee17-4988-9b32-8e4cb1421636.jpg',
 '0010ae4e-0879-4447-9610-e9d422caadd7.jpg',
 '0012b344-1bf9-421b-977e-d4ad8a6dcb7b.jpg',
 '00141b2a-0ed8-45b0-8823-8c0dbef3ac93.jpg',
 '0014f5f6-c37f-4922-8524-efc118f7d17b.jpg',
 '0018f7dd-217b-478b-9858-289922cf9715.jpg']

In [138]:
# Loop through image_paths and upload the image as long as its name isn't in gs_image_paths_name
from tqdm.auto import tqdm

num_images_uploaded = 0
for image_path in tqdm(image_paths):
    if image_path.name not in gs_image_paths_name:
        
        # Upload the image to Google Storage
        upload_to_gs(bucket_name=GS_BUCKET_NAME,
                     source_file_name=image_path,
                     destination_blob_name=f"all_images/{image_path.name}")
        
        num_images_uploaded += 1

print(f"[INFO] Uploaded {num_images_uploaded} images to GCP bucket")

  0%|          | 0/1381 [00:00<?, ?it/s]

[INFO] Uploaded 0 images to GCP bucket


In [139]:
# Track the changes to GCP bucket with Weights & Biases
from utils.wandb_utils import wandb_add_artifact_with_reference

In [141]:
# Track updated images_dir in Weights & Biases
food_images_path_on_gs = "gs://food_vision_bucket_with_object_versioning/all_images"

wandb_add_artifact_with_reference(wandb_run=run,
                                  artifact_name="food_vision_199_classes_images",
                                  artifact_type="dataset",
                                  description="Images for FoodVision project",
                                  reference_path=food_images_path_on_gs)

[INFO] Logging 'food_vision_199_classes_images' from 'gs://food_vision_bucket_with_object_versioning/all_images' to Weights & Biases...


wandb: Generating checksum for up to 1000000000 objects with prefix "all_images"... Done. 10.6s


In [91]:
# Get list of images already in GCP bucket
gs_image_paths = get_list_of_blobs(bucket_name=GS_BUCKET_NAME, prefix="all_images")
print(f"[INFO] There are {len(gs_image_paths)} images in the GCP bucket")

[INFO] There are 25907 images in the GCP bucket


In [ ]:
# Next:
# Create labels CSV for all photos in food_photos ✅
# Add labels CSV to original annotations ✅
# Upload all photos in food_photos to GCP ✅
# Track updates in W&B ✅

# Train a model and track how it performs (by only upgrading training data)